<a href="https://colab.research.google.com/github/tam1444AH/UH-Insure-NSA/blob/main/notebooks/codeLLMFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
%pip install nbstripout

REPO_URL="https://github.com/tam1444AH/UH-Insure-NSA.git"
REPO="UH-Insure-NSA"

os.chdir("/content")

# If repo exists, update it; otherwise, clone fresh
if os.path.exists(REPO):
    print(f"Repo '{REPO}' exists, pulling latest changes...")
    os.chdir(REPO)
    !git reset --hard HEAD   # optional: discard local changes
    !git pull
else:
    print(f"Cloning repo '{REPO}'...")
    !git clone "$REPO_URL" "$REPO"
    os.chdir(REPO)

!nbstripout --install
!git branch -a


# Install dependencies if present
if os.path.exists("requirements.txt"):
    !pip install -r requirements.txt
if os.path.exists("pyproject.toml"):
    !pip install -e .

# save test.

In [ ]:
from model.test import test
test()

In [ ]:
!pip -q install -U huggingface_hub hf_transfer
!export HF_HUB_ENABLE_HF_TRANSFER=1

import os
from google.colab import userdata
from huggingface_hub import login, whoami

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

In [ ]:
from google.colab import auth
auth.authenticate_user()

# project_id = 'code-llm-finetuning'
# !gcloud config set project {project_id}

# GCS_PATH = "gs://code-llm-fine-tuning-security-analytics/data/source_code.jsonl"
# LOCAL_PATH = "/tmp/source_code.jsonl"

# !gsutil ls {GCS_PATH}
# !gsutil -m cp {GCS_PATH} {LOCAL_PATH}

# !ls -lh {LOCAL_PATH}
# !head -n 2 {LOCAL_PATH}

# import json

# examples = [
#     {"filename": "xor.cry", "content": "module Xor where\nproperty xor_inverse (a:[8], b:[8]) = ((a ^^ b) ^^ b) == a"},
#     {"filename": "xor.saw", "content": "cryptol_load \"xor.cry\";\nprove_print z3 {{ Xor::xor_inverse }};"}
# ]

# with open("/tmp/source_code.jsonl", "w") as f:
#     for ex in examples:
#         f.write(json.dumps(ex) + "\n")

# !ls -lh /tmp/source_code.jsonl
# !head -n 2 /tmp/source_code.jsonl



#!gsutil cp gs://code-llm-fine-tuning-security-analytics/data/source_code.jsonl /tmp/source_code.jsonl
#!cat /tmp/source_code.jsonl

In [ ]:
os.chdir("/content")
# !git clone https://github.com/GaloisInc/cryptol-specs.git
# !pip install tqdm

# import os, json, re
# from tqdm import tqdm

# ROOT_DIR = "cryptol-specs"
# OUT_FILE = "/tmp/source_code.jsonl"

# # Heuristic filters (you can tweak these)
# MIN_LEN = 50          # skip tiny examples
# MAX_LEN = 200_000     # skip giant test vectors
# MIN_KEYWORDS = ["module", "property", "where"]
# EXCLUDE_DIRS = ["test", "deprecated", "examples/Regression"]

In [ ]:
# def is_valid_cry(path, content):
#     for ex in EXCLUDE_DIRS:
#         if ex.lower() in path.lower():
#             return False
#     if len(content) < MIN_LEN or len(content) > MAX_LEN:
#         return False
#     if sum(c.isascii() for c in content) / len(content) < 0.8:
#         return False
#     if not any(kw in content for kw in MIN_KEYWORDS):
#         return False
#     return True

# records = []
# for root, _, files in os.walk(ROOT_DIR):
#     for fname in files:
#         if fname.endswith(".cry"):
#             fpath = os.path.join(root, fname)
#             try:
#                 with open(fpath, encoding="utf-8", errors="ignore") as f:
#                     content = f.read()
#                 if is_valid_cry(fpath, content):
#                     records.append({
#                         "filename": os.path.relpath(fpath, ROOT_DIR),
#                         "content": content
#                     })
#             except Exception as e:
#                 print("Error reading", fpath, ":", e)

# with open(OUT_FILE, "w", encoding="utf-8") as out:
#     for rec in records:
#         out.write(json.dumps(rec) + "\n")

# print(f"Done. Wrote {len(records)} Cryptol specs to {OUT_FILE}")
# !ls -lh {OUT_FILE}
# !head -n 3 {OUT_FILE}

In [ ]:
import json, hashlib

src = "/content/verified_GPThybrid.jsonl"
out = "/content/hybrid_tagged_unsupervised.jsonl"

def sha1_of(text):
    return hashlib.sha1(text.encode("utf-8")).hexdigest()[:10]

with open(out, "w") as f_out, open(src, "r") as f_in:
  for line in f_in:
    try:
      data = json.loads(line)
      filetype = data.get("filetype", "").lower()
      filename = data.get("filename", "unknown")
      content = data.get("content", "").strip()
      variant = data.get("variant", "hybrid")

      if filetype == "cry" or filename.endswith(".cry"):
          tag = "cryptol"
      elif filetype == "saw" or filename.endswith(".saw"):
          tag = "saw"
      else:
          tag = "unknown"

      sha = sha1_of(content)
      wrapped = (
          f"<FILE:{tag}> <PATH:{filename}> <VARIANT:{variant}> <SHA:{sha}>\n"
          f"<CODE>\n{content}\n</CODE>\n</FILE>\n"
      )
      f_out.write(json.dumps({"content": wrapped}) + "\n")
    except Exception as e:
      print("skip:", e)



In [ ]:
!pip install -q transformers datasets peft bitsandbytes flash-attn

MODEL = "Qwen/Qwen2.5-Coder-32B"  # Model checkpoint on the Hugging Face Hub
DATASET = "/content/hybrid_tagged_unsupervised.jsonl" #Josh's preproccesed dataset.
DATA_COLUMN = "content"  # Column name containing the code content

SEQ_LENGTH = 2048  # Sequence length

# Training arguments
# MAX_STEPS = 1#2000  # max_steps
MAX_STEPS = 200#2000  # max_steps
BATCH_SIZE = 1  # batch_size
# GR_ACC_STEPS = 16  # gradient_accumulation_steps
GR_ACC_STEPS = 8  # gradient_accumulation_steps
# LR = 5e-4  # learning_rate
LR = 2e-4                     # a bit lower for stability
LR_SCHEDULER_TYPE = "cosine"  # lr_scheduler_type
WEIGHT_DECAY = 0.05  # weight_decay
# WEIGHT_DECAY = 0.01  # weight_decay
NUM_WARMUP_STEPS = 100  # num_warmup_steps
# NUM_WARMUP_STEPS = 30  # num_warmup_steps
# EVAL_FREQ = 200
EVAL_FREQ = 50
SAVE_FREQ = 200
LOG_FREQ = 10
LOG_FREQ = 50
# EVAL_FREQ = 100  # eval_freq
# SAVE_FREQ = 100  # save_freq
# LOG_FREQ = 25  # log_freq
OUTPUT_DIR = "peft-FT-2.5-Coder-32b"  # output_dir
BF16 = True  # bf16
FP16 = False  # no_fp16

# FIM trasformations arguments
FIM_RATE = 0.5  # fim_rate
FIM_SPM_RATE = 0.5  # fim_spm_rate

# LORA
LORA_R = 8  # lora_r
LORA_ALPHA = 32  # lora_alpha
LORA_DROPOUT = 0.05  # lora_dropout
LORA_TARGET_MODULES = "c_proj,c_attn,q_attn,c_fc,c_proj"  # lora_target_modules

# bitsandbytes config
USE_NESTED_QUANT = True  # use_nested_quant
BNB_4BIT_COMPUTE_DTYPE = "bfloat16"  # bnb_4bit_compute_dtype

SEED = 0

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed,
    BitsAndBytesConfig,
)

set_seed(SEED)

In [ ]:
from datasets import load_dataset

DATASET = "/content/hybrid_tagged_unsupervised.jsonl"
DATA_COLUMN = "content"

dataset = load_dataset("json", data_files={"train": DATASET})
train_data = dataset["train"]

# quick sanity check: make sure the field exists
print(train_data[0].keys())
assert DATA_COLUMN in train_data.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"


In [ ]:
import torch
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

def chars_token_ratio(dataset, tokenizer, data_column, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """

    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        total_characters += len(example[data_column])
        total_tokens += len(tokenizer(example[data_column]).tokens())

    return total_characters / total_tokens


chars_per_token = chars_token_ratio(train_data, tokenizer, DATA_COLUMN)
print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")


In [ ]:
from model.dataset.util import *

In [ ]:
from model.dataset.constantLengthDataset import ConstantLengthDataset


train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        infinite=True,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)
eval_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        #valid_data,
        infinite=False,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.tuners.lora import LoraLayer

load_in_8bit = False

# 4-bit quantization
compute_dtype = getattr(torch, BNB_4BIT_COMPUTE_DTYPE)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=USE_NESTED_QUANT,
)

device_map = "auto"#{"": 0}

model = AutoModelForCausalLM.from_pretrained(
        MODEL,
        load_in_8bit=load_in_8bit,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map=device_map,
        use_cache=False,  # We will be using gradient checkpointing
        trust_remote_code=True,
        attn_implementation="flash_attention_2",
        offload_folder="/content/offload"
)

model = prepare_model_for_kbit_training(model)

In [ ]:
blk = model.model.layers[0]           # Llama/Qwen-style
print("ATTN:", blk.self_attn)         # has q_proj, k_proj, v_proj, o_proj
print("MLP:", blk.mlp)
target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

In [ ]:
# Set up lora
peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
train_data.start_iteration = 0


training_args = TrainingArguments(
    output_dir=f"tam1444AH/{OUTPUT_DIR}",
    dataloader_drop_last=True,
    eval_strategy="steps",
    save_strategy="steps",
    max_steps=MAX_STEPS,
    eval_steps=EVAL_FREQ,
    save_steps=SAVE_FREQ,
    logging_steps=LOG_FREQ,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_steps=NUM_WARMUP_STEPS,
    gradient_accumulation_steps=GR_ACC_STEPS,
    gradient_checkpointing=True,
    fp16=FP16,
    bf16=BF16,
    weight_decay=WEIGHT_DECAY,
    push_to_hub=True,
    include_tokens_per_second=True,
    report_to=[]
)
print(f"Training samples: {len(train_data)}")
print(f"Sample text:\n\n{train_data[0][DATA_COLUMN][:400]}")

In [ ]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

print("Training...")
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
!pip install -q transformers peft

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load the base model
# base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-Coder-32B", trust_remote_code=True)

# Load your LoRA fine-tuned adapter on top of it
model = PeftModel.from_pretrained(model, "tam2003/peft-FT-2.5-Coder-32b")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-32B", trust_remote_code=True)

# Quick test
# prompt = "Write a Cryptol specification for a function xor_inverse that proves (a XOR b) XOR b == a"
# inputs = tokenizer(prompt, return_tensors="pt")
# outputs = model.generate(**inputs, max_new_tokens=150)

# print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
from transformers import pipeline
import gc, torch
gc.collect()
torch.cuda.empty_cache()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="balanced_low_0",  # spreads layers more evenly
    dtype="bfloat16",        # uses half precision
)

prompt = """
Translate this English spec into Cryptol:

"A function sha256_hash that takes a 512-bit input block and returns a 256-bit digest."
"""
result = pipe(prompt, max_new_tokens=300)
print(result[0]["generated_text"])

prompts = [
    'Translate this English spec into Cryptol:\n\n"A function xor_inverse that proves (a XOR b) XOR b == a"',
    'Translate this English spec into Cryptol:\n\n"A function sha256_hash that takes a 512-bit input and returns a 256-bit digest"',
    'Translate this English spec into Cryptol:\n\n"Create a property proving that addition is commutative for 8-bit words."'
]

for p in prompts:
    print("\n=== Prompt ===")
    print(p)
    inputs = tokenizer(p, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150)
    print("\n--- Generated Cryptol ---")
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
!apt-get update -q
!apt-get install -y wget tar

!wget -q https://github.com/GaloisInc/saw-script/releases/download/v1.3/saw-1.3-ubuntu-22.04-X64-with-solvers.tar.gz
!tar -xzf saw-1.3-ubuntu-22.04-X64-with-solvers.tar.gz
!mv saw-1.3-ubuntu-22.04-X64-with-solvers saw

!wget -q https://github.com/GaloisInc/cryptol/releases/download/3.3.0/cryptol-3.3.0-ubuntu-20.04-X64-with-solvers.tar.gz
!tar -xzf cryptol-3.3.0-ubuntu-22.04-X64-with-solvers.tar.gz
!mv cryptol-3.3.0-ubuntu-22.04-X64-with-solvers cryptol

In [ ]:
import os
os.environ["PATH"] = "/content/saw/bin:/content/cryptol/bin:" + os.environ["PATH"]

In [ ]:
!saw --version
!cryptol --version

In [ ]:
!git clone https://github.com/GaloisInc/cryptol-specs.git

In [ ]:
%cd /content/cryptol-specs/Primitive/Symmetric/Cipher/Block/AES/Verifications
os.environ["CRYPTOLPATH"] = "/content/cryptol-specs"

In [ ]:
!saw AES128.saw

In [ ]:
%cd /content
!git clone https://github.com/plobethus/crypto-c.git

In [ ]:
%cd crypto-c

In [ ]:
!chmod +x scripts/colab_setup.sh
!bash scripts/colab_setup.sh
%cd /content